In [1]:
#-*- coding : utf-8 -*-
# coding: utf-8

%load_ext autoreload
%autoreload 2
%matplotlib notebook

# 0. what for
1. 用于生成人工数据集: target data
2. v2是最终的版本
# 1. preparations
## 1.1 import and data path

In [8]:
import pandas as pd
import random
from tqdm import tqdm
import numpy as np
from GT_model.GT_2.SA_for_PT_funcs_delta_eq1 import *
from matplotlib import pyplot as plt
import plotly.graph_objs as go

settings_small_NN_path = r"../data/small_settings_NN.csv"
settings_large_NN_path = r'E:\DATA\large_dta\large_settings_NN.csv'

# Sampled params and k
params_and_K_sampled_path = r"../data/auction_assign.csv"


NOISE = True                    # 是否添加noise
noise_pct = 0.05                # 噪音占比:我们希望生成的data总体上最多浮动的百分比noise_pct

if NOISE:
    target_root_path= "../data/artificial_targets_v2_"+"noise="+str(noise_pct)+"/"
    target_ls_T_root_path= "../data/artificial_targets_v2_"+"noise="+str(noise_pct)+"_ls_T"+"/"
else:
    target_root_path= "../data/artificial_targets_v2_noise=0/"
    target_ls_T_root_path= "../data/artificial_targets_v2_noise=0_ls_T/"

target_file_head = "target_data_NP_"
target_file_tail= ".csv"

bound_noise_duration = [0]      # no upper bound

## 1.2 read in

In [9]:
data_key_small = pd.read_csv(settings_small_NN_path, encoding="utf-8")
data_key_large = pd.read_csv(settings_large_NN_path, encoding="utf-8")
data_key = pd.concat([data_key_small,data_key_large],axis=0,ignore_index=True)
len_settings = data_key.shape[0]        # 一共要sample len_settings 这么多settings

params_and_K = pd.read_csv(params_and_K_sampled_path, encoding="utf-8")
print(f"一共需要sample {len_settings} 个不同的settings")

一共需要sample 1276 个不同的settings


# 2. sample

In [10]:
def draw_hist(data, title ):
    # create a histogram trace
    trace = go.Histogram(x=data, nbinsx=300,histnorm='probability')
    # create a layout
    layout = go.Layout(
        title= title,
        xaxis=dict(title='N'),
        yaxis=dict(title='Frequency')
    )
    # create a figure
    fig = go.Figure(data=[trace], layout=layout)
    # show the plot
    return fig


In [11]:
for i in tqdm(range(len_settings)):
# for i in tqdm(range(6)):
# for i in tqdm(range(3,4)):

    v = data_key.loc[i,'retail'].item()
    b = data_key.loc[i,'bidfee'].item()
    d = data_key.loc[i,'bidincrement'].item()

    GT_choice = params_and_K.loc[i,'GT_choice'].item()
    observations = params_and_K.loc[i,'observations'].item()    # 一共有这么多records

    # d==0 suggests a fixed-price auction
    if d == 0:
        # T = np.inf                                             # duration limitation
        T = int(observations)                                    # duration limitation
    else:
        T = int(np.floor((v-b)/d))                         # duration limitation

    if GT_choice==0:
        # for GT-1
        U_1 = get_U_GT1(T,v,d,b,eps=0.)
        P_1 = get_P(U_1,T)
        P = P_1/P_1.sum()
    else:
        # for GT-2
        alpha = params_and_K.loc[i,'alpha'].item()
        labda = params_and_K.loc[i,'labda'].item()
        U_2 = get_U_GT2(T,v,d,b,alpha,labda,eps=0.)
        P = get_P(U_2,T)

    # 生成N
    # 1<= possible_duration <= T
    possible_duration = np.arange(1,T+1)        # list of all possible N, 从这里sample out
    assert len(P)==len(possible_duration),"P和N不等长"
    # 根据P抽样observations个records
    sampled_N = np.random.choice(possible_duration, size=observations, p=P)

    # fig = draw_hist(sampled_N, "without noise mu="+str(np.mean(sampled_N))+" std=" +str(np.std(sampled_N)))
    # fig.show()

    # Add some noises which are set between bound of 'bound_noise_duration'.
    # Add Gaussian noise to each of the data point.
    mu_tmp = noise_pct*np.mean(sampled_N)
    # mu_tmp = 0
    std_tmp = noise_pct*np.std(sampled_N)
    # sampled_N_noise = np.maximum(np.random.normal(mu_tmp, std_tmp, size=observations),bound_noise_duration[0])
    sampled_N_noise = np.random.normal(mu_tmp, std_tmp, size=observations)

    sampled_N_noise = sampled_N_noise.astype(int)       # 设为int
    sampled_N_wnoise = np.where((sampled_N + sampled_N_noise)>0, sampled_N + sampled_N_noise, sampled_N)
    # 只保留加完可以保证整体>0的noise
    # sampled_N += sampled_N_noise
    # Sort out

    # fig = draw_hist(sampled_N_wnoise, "with noise mu="+str(np.mean(sampled_N_wnoise))+" std=" +str(np.std(sampled_N_wnoise)))
    # fig.show()

    # # Save
    if NOISE:
        sampled_N_wnoise.sort()
        N_pd = pd.DataFrame(sampled_N_wnoise,columns=['N'])
    else:
        sampled_N.sort()
        N_pd = pd.DataFrame(sampled_N,columns=['N'])

    output_path = target_root_path + target_file_head + str(i).zfill(4) + target_file_tail
    N_pd.to_csv(output_path,header=True,index=False,encoding="utf-8")

    # 只保留N<=T的target data
    N_pd_ls_T = N_pd[N_pd.N<=T]
    output_ls_T_path = target_ls_T_root_path + target_file_head + str(i).zfill(4) + target_file_tail
    N_pd_ls_T.to_csv(output_ls_T_path, encoding="utf-8", header=True, index=False)

print("Done")

100%|██████████| 1276/1276 [02:35<00:00,  8.22it/s]

Done


- 只保留N<=T的target data